In [1]:
from fastai import *
from fastai.text import *

In [2]:
from bs4 import BeautifulSoup

In [3]:
base_path = "https://www.aozora.gr.jp/"
page = requests.get(base_path + "index_pages/person1779.html")

In [4]:
soup = BeautifulSoup(page.text, 'html')
texts = []
for a in soup.find_all("a"):
    href = a.get("href")
    if isinstance(href, str) and href.startswith("../cards"):
        tmp_path = base_path + "/".join(href.split("/")[1:])
        tmp_page = requests.get(tmp_path)
        tmp_soup = BeautifulSoup(tmp_page.text, "html.parser")
        for tmp_a in tmp_soup.find_all("a"):
            tmp_href = tmp_a.get("href")
            if (
                isinstance(tmp_href, str) and
                tmp_href.startswith("./files") and
                tmp_href.endswith(".html")
            ):
                source_path = "/".join(tmp_path.split("/")[:-1] + [tmp_href[2:]])
                source_page = requests.get(source_path)
                source_page.encoding = "shift-jis"
                source_soup = BeautifulSoup(source_page.text, "html.parser")
                text = (
                    source_soup
                    .find("div", {"class": "main_text"})
                    .text
                    .strip()
                    .replace("\n", "")
                    .replace("\r", "")
                    .replace("\u3000", "")
                )
                texts.append(text)

In [5]:
splitted_texts = [ts + "。" for t in texts for ts in t.split("。")]
df = pd.DataFrame(splitted_texts).drop_duplicates().reset_index(drop=True)

In [6]:
df.head()

,0
0,１あるにちよう日のごご、丹下（たんげ）サト子ちゃんと、木村（きむら）ミドリちゃんと、野崎（の...
1,三人とも、小学校三年生のなかよしです。
2,「あらっ。
3,」サト子ちゃんが、なにを見たのか、ぎょっとしたようにたちどまりました。
4,ミドリちゃんもサユリちゃんもびっくりして、サト子ちゃんの見つめている方をながめました。


In [7]:
import re
from natto import MeCab

In [19]:
nm = MeCab()

class MeCabTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = 'ja'
    def add_special_cases(self, toks:Collection[str]): pass
    def tokenizer(self,raw_sentence):
        return [node.surface for node in nm.parse(raw_sentence, as_nodes=True)]

In [20]:
tokenizer = Tokenizer(MeCabTokenizer, 'ja')
processor = [
    TokenizeProcessor(tokenizer=tokenizer),
    NumericalizeProcessor(max_vocab=60000,min_freq=2)
]

data_lm = (
    TextList
    .from_df(df,Path("data"),cols=[0],processor=processor)
    .split_by_rand_pct(0.1)
    .label_for_lm()
    .databunch(bs=64)
)

In [21]:
data_lm.show_batch()

idx,text
0,歩い て い まし た 。 xxbos 三 人 とも 、 小学校 三 年生 の なかよし です 。 xxbos 「 あらっ 。 xxbos 」 サト子 ちゃん が 、 なに を 見 た の か 、 ぎょっと し た よう に たちどまり まし た 。 xxbos ミドリ ちゃん も サユリ ちゃん も びっくり し て 、 サト子 ちゃん の 見つめ て いる 方 を ながめ まし た 。 xxbos する と
1,の 庭 に 椅子 を 置い て 、 この 方面 から の 侵入 者 を 防ぐ という 手筈 で あっ た 。 xxbos 早い 夕食 を 済ませ て 、 一同 部署 に つい た が 、 川手 氏 は それでも まだ 安心 し きれ ぬ 体 （ てい ） で 、 妙子 さん の 部屋 に 入っ たり 出 たり し ながら 、 廊下 の 宗像 博士 の 前 を
2,と し て いる の だ 。 xxbos 目 を 閉 （ ふさ ） い で も 、 断末魔 の 悲痛 な うめき声 が 聞え て 来る 。 xxbos しかも それ は 、 一寸 だ めし 五 分 だ めし 、 歌舞伎 （ かぶ き ） 芝居 の 殺し 場 そっくり の 、 あの いやらしい 、 陰惨 な 、 xxunk （ そく そく ） として 鬼気 の 身 に 迫る
3,が 、 果し合い でも する よう な 顔 で 突 立っ て い た 。 xxbos 立 上る 時 、 興奮 の 余り 、 つい 椅子 を 倒し た の で ある 。 xxbos 論理 と 空想 の 外 に は 、 現実 の 証拠 という もの が 一つ も ない じゃ ない か 。 xxbos 証拠 を 得 よう に も 、 残念 ながら 竜子 が 死ん で しまっ
4,間違っ た 推断 によって 罪 （ つみ ） せら れ ん と する 我 学界 の 長者 を 救う もの は 、 偶然 に も その 現場 に xxunk （ い あわ ） し て 、 一寸 し た 証拠 物件 を 手 に 入れ た 、 この 私 の 外 に ない と 信ずる が 故に 、 当然 の 義務 として 、 この 挙 に 出 （ い ）


In [22]:
learn = language_model_learner(data_lm, AWD_LSTM, pretrained=False)

In [ ]:
learn.lr_find()
learn.recorder.plot()

epoch,train_loss,valid_loss,accuracy,time


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(16, 1e-02)

In [ ]:
learn.predict("二十面相", n_words=50)

In [ ]:
learn.export("edogawa.pkl")